# Multiple Single line sections
## Single line section *StartSection*
> - Start *Before* `StartSection`
> - End *Before* `EndSection`

In [ ]:
sub_section1 = Section(
    name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before')
    )

test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section1.read(test_iter))
pprint(sub_section1.read(test_iter))
pprint(sub_section1.read(test_iter))

['StartSection A']
['StartSection B']
[]


Single line `StartSections` are returned
|Expected|Actual|
|-|-|
|`['StartSection Name: A']`|`['StartSection Name: A']`|
|`['StartSection Name: B']`|`['StartSection Name: B']`|
|`[]`|`[]`|

### Defining a section that calls `sub_section1`

In [ ]:
full_section = Section(
    name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    processor=[sub_section1]
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection A'], ['StartSection B']]


Both *StartSection* lines are found.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
        [<br>
          ['StartSection Name: A'],<br>
          ['StartSection Name: B'],<br>
        ]</code></td>
        <td><code>
        [<br>
          ['StartSection Name: A'],<br>
          ['StartSection Name: B'],<br>
        ]</code></td></tr>
</table>

### Multiple Attempts to define Single line *EndSection* 
#### Attempt #1. Copy *StartSection* design
> - Start *Before* `EndSection`
> - End *Before* `StartSection`


In [ ]:
sub_section2a = Section(
    name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2a.read(test_iter))
pprint(sub_section2a.read(test_iter))
pprint(sub_section2a.read(test_iter))

['EndSection A']
['EndSection B', 'More text to be ignored']
[]


The result is expected, but not desired.
- This doesn't work well because there isn't another *StartSection* after the last it *EndSection*.
- As a result, it includes the unwanted text after the last *EndSection*.
 
|Expected|Actual|
|-|-|
|`['EndSection A']`|`['EndSection A']`|
|`['EndSection B', 'More text to be ignored']`|`['EndSection B', 'More text to be ignored']`|
|`[]`|`[]`|

#### Attempt #2
> - Start *Before* `EndSection`
> - End *After* `EndSection`

- This doesn't work because The first line is not tested so it only breaks after the second occurrence of *EndSection*.

In [ ]:
sub_section2b = Section(
    name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2b.read(test_iter))
pprint(sub_section2b.read(test_iter))
pprint(sub_section2b.read(test_iter))

['EndSection A', 'StartSection B', 'EndSection B']
[]
[]



|Expected|Actual|
|-|-|
|`['EndSection A', 'StartSection B', 'EndSection B']`|`['EndSection A', 'StartSection B', 'EndSection B']`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Attempt #3
> - Start *Before* `EndSection`
> - End *After* `EndSection`
> - Enable testing of first item

- This should work because the first line should be tested and trigger a break.

In [ ]:
sub_section2c = Section(
    name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))

['EndSection A']
['EndSection B']
[]




|Expected|Actual|
|-|-|
|`['EndSection A']`|`['EndSection A']`|
|`['EndSection B']`|`['EndSection B']`|
|`[]`|`[]`|


#### Attempt #3.5
> - Start *Before* `EndSection`
> - End *Before* `EndSection`
> - Enable testing of first item

- This should __<u>not</u>__ work because the first line should be tested and 
  trigger a break before returning anything.

In [ ]:
sub_section2cc = Section(
    name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2cc.read(test_iter))
pprint(sub_section2cc.read(test_iter))
pprint(sub_section2cc.read(test_iter))

[]
[]
[]




|Expected|Actual|
|-|-|
|`[]`|`[]`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Attempt #4
> - Start *Before* `EndSection`
> - End is `True` (Always Break)
> - Enable testing of first item

- This should work because the first line should be tested and always trigger a break.

In [ ]:
sub_section2d = Section(
    name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2d.read(test_iter))
pprint(sub_section2d.read(test_iter))
pprint(sub_section2d.read(test_iter))

['EndSection A']
['EndSection B']
[]




|Expected|Actual|
|-|-|
|`['StartSection A']`|`['EndSection A']`|
|`['StartSection B']`|`['EndSection B']`|
|`[]`|`[]`|

#### Attempt #5
> - Start *Before* `EndSection`
> - End is `True` (Always Break)
> - Don't enable testing of first item

- This should <u>not</u> work because the break should trigger on the second line.

In [ ]:
sub_section2e = Section(
    name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2e.read(test_iter))
pprint(sub_section2e.read(test_iter))
pprint(sub_section2e.read(test_iter))

['EndSection A', 'StartSection B']
['EndSection B', 'More text to be ignored']
[]




|Expected|Actual|
|-|-|
|`['EndSection A', 'StartSection B']`|`['EndSection A', 'StartSection B']`|
|`['EndSection B', 'More text to be ignored']`|`['EndSection B', 'More text to be ignored']`|
|`[]`|`[]`|

**`end_section=SectionBreak(True)` defaults to `break_offset='Before'`**

#### Attempt #6
> - Start *Before* `EndSection`
> - End *After* ___`True`___ (Always Break)
> - Don't enable testing of first item

- This should <u>not</u> work because the break should trigger *After* the second line.

In [ ]:
sub_section2f = Section(
    name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2f.read(test_iter))
pprint(sub_section2f.read(test_iter))
pprint(sub_section2f.read(test_iter))

['EndSection A', 'StartSection B']
['EndSection B', 'More text to be ignored']
[]



|Expected|Actual|
|-|-|
|`['EndSection A', 'StartSection B']`|`['EndSection A', 'StartSection B']`|
|`['EndSection B', 'More text to be ignored']`|`['EndSection B', 'More text to be ignored']`|
|`[]`|`[]`|

#### Attempt #7
> - Start *Before* `EndSection`
> - Enable testing of first item
> - No End setting

- This should start with the first *EndSection* line and not stop until the end of the iterator.

In [ ]:
sub_section2g = Section(
    name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2g.read(test_iter))
pprint(sub_section2g.read(test_iter))
pprint(sub_section2g.read(test_iter))

['EndSection A', 'StartSection B', 'EndSection B', 'More text to be ignored']
[]
[]



|Expected|Actual|
|-|-|
|`['EndSection Name: A', ['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Single line section *EndSection* __Final Design__.
> - Start *Before* `EndSection`
> - End *Before* ___`True`___ (Always Break)
> - Don't enable testing of first item

- This should work because the break should trigger *Before* the second line.

In [ ]:
sub_section2 = Section(
    name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2.read(test_iter))
pprint(sub_section2.read(test_iter))
pprint(sub_section2.read(test_iter))

['EndSection A']
['EndSection B']
[]


End breaks *Before* second line. 

|Expected|Actual|
|-|-|
|`['EndSection Name: A']`|`['EndSection Name: A']`|
|`['EndSection Name: B']`|`['EndSection Name: B']`|
|`[]`|`[]`|

In [ ]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B', 
    'More text to be ignored',
    ]

In [ ]:
start_sub_section = Section(
    name='StartSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before')
    )
end_sub_section = Section(
    name='EndSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )


### Combined Start and End subsections Single line section 
***sub_section1***
> - Start *Before* `StartSection`
> - End *Before* `EndSection`

***sub_section2*** 
> - Start *Before* `EndSection`
> - End *Before* ___`True`___ (Always Break)
> - Don't enable testing of first item

`processor=[[sub_section1, sub_section2]]`

In [ ]:

full_section = Section(
    name='Full',
    processor=[[start_sub_section, end_sub_section]]
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[{'EndSection': ['EndSection A'], 'StartSection': ['StartSection A']},
 {'EndSection': ['EndSection B'], 'StartSection': ['StartSection B']}]


List of dictionaries. 
One dictionary for each section group.
<table>
    <thead><tr><th>Expected</th><th>Actual</th></tr></thead>
        <tr>
            <td><code>
                [<br>  
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},<br>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']}<br>
                ]
            </code></td>
            <td><code>
                [<br>  
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},<br>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']}<br>
                ]
            </code></td>
        </tr>
</table>

### Defining a Top Section that calls two single line subsections.

#### Two Subsection definitions 

In [ ]:
start_sub_section = Section(
    name='StartSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )
end_sub_section = Section(
    name='EndSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )


#### Defining ***top_section*** 
- Contains an ending break:
    > `end_section=SectionBreak('ignored', break_offset='Before')`.

In [ ]:

top_section = Section(
    name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    processor=[[start_sub_section, end_sub_section]]
    )
pprint(top_section.read(GENERIC_TEST_TEXT))

[{'EndSection': ['EndSection A'], 'StartSection': ['StartSection A']},
 {'EndSection': ['EndSection B'], 'StartSection': ['StartSection B']}]


List of dictionaries. 
One dictionary for each section group.
<table>
    <thead><tr><th>Expected</th><th>Actual</th></tr></thead>
        <tr>
            <td><code>
                [<br>  
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},<br>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']}<br>
                ]
            </code></td>
            <td><code>
                [<br>  
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},<br>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']}<br>
                ]
            </code></td>
        </tr>
</table>

#### Adding subsection C and subsection D

In [ ]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B', 
    'StartSection C',
    'EndSection C',
    'StartSection D',
    'EndSection D',
    'More text to be ignored',
    ]

pprint(top_section.read(GENERIC_TEST_TEXT))

[{'EndSection': ['EndSection A'], 'StartSection': ['StartSection A']},
 {'EndSection': ['EndSection B'], 'StartSection': ['StartSection B']},
 {'EndSection': ['EndSection C'], 'StartSection': ['StartSection C']},
 {'EndSection': ['EndSection D'], 'StartSection': ['StartSection D']}]


List of dictionaries. 
One dictionary for each section group.
<table>
    <thead><tr><th>Expected</th><th>Actual</th></tr></thead>
        <tr><td><code>[</code></td><td><code>[</code></td></tr>
        <tr>
            <td><code>
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},
            </code></td>
        </tr><tr>
            <td><code>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']},
            </code></td>
        </tr><tr>
            <td><code>
                {'StartSection': ['StartSection C'],<br>
                'EndSection':   ['EndSection C']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection C'],<br>
                'EndSection':   ['EndSection C']},
            </code></td>
        </tr><tr>
            <td><code>
                {'StartSection': ['StartSection D'],<br>
                'EndSection':   ['EndSection D']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection D'],<br>
                'EndSection':   ['EndSection D']},
            </code></td>
        </tr>
        <tr><td><code>]</code></td><td><code>]</code></td></tr>
</table>

### Including unwanted text in between subsections

In [ ]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',  # Missing 'EndSection B',
    'StartSection C',
    'EndSection C',
    'More text to be ignored',   # 'ignored' triggers end of top section
    'StartSection D',
    'EndSection D',
    'More text to be ignored',
    ]

pprint(top_section.read(GENERIC_TEST_TEXT))


[{'EndSection': ['EndSection A'], 'StartSection': ['StartSection A']},
 {'EndSection': ['EndSection C'], 'StartSection': ['StartSection B']}]


List of dictionaries stops after
 _'More text to be ignored'_ line.
<table>
    <thead><tr><th>Expected</th><th>Actual</th></tr></thead>
        <tr><td><code>[</code></td><td><code>[</code></td></tr>
        <tr>
            <td><code>
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},
            </code></td>
        </tr><tr>
            <td><code>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']},
            </code></td>
        </tr><tr>
            <td><code>
                {'StartSection': ['StartSection C'],<br>
                'EndSection':   ['EndSection C']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection C'],<br>
                'EndSection':   ['EndSection C']},
            </code></td>
        </tr>
        <tr><td><code>]</code></td><td><code>]</code></td></tr>
</table>

#### Including unwanted text in between the start and end of subsection C

In [ ]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B', 
    'StartSection C',
    'More text to be ignored',   # 'ignored' triggers end of top section
    'EndSection C',
    'StartSection D',
    'EndSection D',
    'More text to be ignored',
    ]

pprint(top_section.read(GENERIC_TEST_TEXT))

[{'EndSection': ['EndSection A'], 'StartSection': ['StartSection A']},
 {'EndSection': ['EndSection B'], 'StartSection': ['StartSection B']},
 {'StartSection': ['StartSection C']}]


List of dictionaries stops after
 _'More text to be ignored'_ line.
 For Section __C__, only _'StartSection'_ is in the dictionary.
<table>
    <thead><tr><th>Expected</th><th>Actual</th></tr></thead>
        <tr><td><code>[</code></td><td><code>[</code></td></tr>
        <tr>
            <td><code>
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},
            </code></td>
        </tr><tr>
            <td><code>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']},
            </code></td>
        </tr><tr>
            <td><code>
                {'StartSection': ['StartSection C']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection C']},
            </code></td>
        </tr>
        <tr><td><code>]</code></td><td><code>]</code></td></tr>
</table>

#### Dropping the corresponding *EndSection* for a *StartSection*.

In [ ]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',  # Missing 'EndSection B',
    'StartSection C',
    'EndSection C',
    'StartSection D',
    'EndSection D',
    'More text to be ignored',
    ]

pprint(top_section.read(GENERIC_TEST_TEXT))


[{'EndSection': ['EndSection A'], 'StartSection': ['StartSection A']},
 {'EndSection': ['EndSection C'], 'StartSection': ['StartSection B']},
 {'EndSection': ['EndSection D'], 'StartSection': ['StartSection D']}]


_'StartSection B'_ is read, then section ends. 
`top_section` then ignores all lines until start of next `end_sub_section`:
_'EndSection C'_.
<style type="text/css">
.hl{font-weight:bold;color:cyan;font-size: 110%;font-family: serif}
</style>
<table>
    <thead><tr><th>Expected</th><th>Actual</th></tr></thead>
        <tr><td><code>[</code></td><td><code>[</code></td></tr>
        <tr>
            <td><code>
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},
            </code></td>
        </tr><tr>
            <td><code>
                {'StartSection': ['StartSection <Span class="hl">B</Span>'],<br>
                'EndSection':   ['EndSection <Span class="hl">C</Span>']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection <Span class="hl">B</Span>'],<br>
                'EndSection':   ['EndSection <Span class="hl">C</Span>']},
            </code></td>
        </tr><tr>
            <td><code>
                {'StartSection': ['StartSection D'],<br>
                'EndSection':   ['EndSection D']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection D'],<br>
                'EndSection':   ['EndSection D']},
            </code></td>
        </tr>
        <tr><td><code>]</code></td><td><code>]</code></td></tr>
</table>

#### Dropping the corresponding *StartSection* for an *EndSection*.

In [ ]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',  
    'EndSection B',  # Missing 'StartSection C',
    'EndSection C',
    'StartSection D',
    'EndSection D',
    'More text to be ignored',
    ]

pprint(top_section.read(GENERIC_TEST_TEXT))


[{'EndSection': ['EndSection A'], 'StartSection': ['StartSection A']},
 {'EndSection': ['EndSection B'], 'StartSection': ['StartSection B']},
 {'EndSection': ['EndSection D'], 'StartSection': ['StartSection D']}]


The entire _Section C_ is skipped because it 
will not look for an _EndSection_ until a _StartSection_ is found.
<table>
    <thead><tr><th>Expected</th><th>Actual</th></tr></thead>
        <tr><td><code>[</code></td><td><code>[</code></td></tr>
        <tr>
            <td><code>
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection A'],<br>
                'EndSection':   ['EndSection A']},
            </code></td>
        </tr><tr>
            <td><code>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection B'],<br>
                'EndSection':   ['EndSection B']},
            </code></td>
        </tr><tr>
            <td><code>
                {'StartSection': ['StartSection D'],<br>
                'EndSection':   ['EndSection D']},
            </code></td>
            <td><code>
                {'StartSection': ['StartSection D'],<br>
                'EndSection':   ['EndSection D']},
            </code></td>
        </tr>
        <tr><td><code>]</code></td><td><code>]</code></td></tr>
</table>